In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import os
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
def load_data():
    netflix_csv_file = open("netflix_rating.csv", mode = "w")
    rating_files = ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt']
    for file in rating_files:
        with open(file) as f:
            for line in f:
                line = line.strip()
                if line.endswith(":"):
                    movie_id = line.replace(":", "")
                else:
                    row_data = []
                    row_data = [item for item in line.split(",")]
                    row_data.insert(0, movie_id)
                    netflix_csv_file.write(",".join(row_data))  
                    netflix_csv_file.write('\n')
                    
    netflix_csv_file.close()
    df = pd.read_csv('netflix_rating.csv', sep=",", names = ["movie_id","customer_id", "rating", "date"])
    return df
netflix_rating_df = load_data()
netflix_rating_df.head()

,movie_id,customer_id,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [2]:
netflix_rating_df.drop_duplicates(inplace=True)

In [3]:
from pandas.api.types import CategoricalDtype
from scipy import sparse

users = netflix_rating_df["customer_id"]
movies = netflix_rating_df["movie_id"]
shape = (2649430, 17771)
print(shape,netflix_rating_df["rating"].size )

coo = sparse.coo_matrix((netflix_rating_df["rating"], (users, movies)), shape=shape)
csr = coo.tocsr()

(2649430, 17771) 100480507


In [4]:
csr[1488844,1]

3

In [5]:
csr.shape

(2649430, 17771)

In [6]:
from sklearn.decomposition import NMF
import numpy as np
nmf = NMF(n_components=3, init='random', random_state=0)
W = nmf.fit_transform(csr)
H = nmf.components_
# nR = np.dot(W,H)
# print(nR)

In [7]:
W.shape #user

(2649430, 3)

In [8]:
W

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       ...,
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [1.80649668, 0.94484317, 0.00727281]])

In [9]:
H.shape #movie

(3, 17771)

## Prediksi Data Probe.txt

In [10]:
probe_mov_id = []
probe_user_id = []

f = open("probe.txt", "r")
movie_id = ""

for x in f:
    if x.strip()[-1] == ":":
        movie_id = int(x.strip()[:-1])
    else:
        cust_id = int(x)
        probe_mov_id.append(movie_id)
        probe_user_id.append(cust_id)
        
f.close()

In [11]:
len(probe_user_id)

1408395

In [12]:
len(probe_mov_id)

1408395

In [13]:
y_pred = []
y_actual = []
for mov,user in zip(probe_mov_id,probe_user_id):
    y_pred.append(W[user]@H[:,mov])
    y_actual.append(csr[user,mov])
    

In [14]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_actual, y_pred, squared=False)

In [15]:
rms

3.4596167909989295

## Prediksi Data train

In [ ]:
y_pred = []
y_actual = []
for mov,user in zip(movies,users):
    y_pred.append(W[user]@H[:,mov])
    y_actual.append(csr[user,mov])

In [ ]:
rms = mean_squared_error(y_actual, y_pred, squared=False)
rms